<a href="https://colab.research.google.com/github/jeffersondemota/projeto_/blob/main/comparador(v_14).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import files
uploaded = files.upload()

Saving logo-branca-qfpsii0s3y2kjypwqu57rvsp16k4hj6dc0rz3dj1ia.png to logo-branca-qfpsii0s3y2kjypwqu57rvsp16k4hj6dc0rz3dj1ia (2).png


In [ ]:
# =========================================================
# 🧭 Sistema de Comparação de Custos Logísticos
# Frota Própria x Agregada x CrossDocking
# Executável direto no Google Colab via Streamlit + ngrok
# =========================================================

# === 1️⃣ Criação dos arquivos app.py e run_in_colab.py ===

app_code = r'''
import os
import io
import math
import json
import time
import base64
import requests
import pandas as pd
import numpy as np
import streamlit as st
from datetime import datetime, timedelta
from PIL import Image # Import Pillow library to handle images
import altair as alt # Import altair with alias alt
import sys, subprocess
from math import sqrt
import re # Import regex module

# --- Funções auxiliares (resumidas para clareza) ---
def safe_float(x, default=0.0):
    try: return float(str(x).replace(",", "."))
    except: return default

def to_hours(x):
    if pd.isna(x): return 0
    s=str(x)
    if ":" in s:
        h,m,*_ = s.split(":")
        return float(h)+float(m)/60
    try:
        # Assume numerical input is in minutes and convert to hours
        return float(s) / 60
    except:
        return 0

def _money_to_float(x):
    """
    Converte strings de moeda para float, aceitando formatos:
    'R$ 3.120,00' | '3.120,00' | '3120.00' | 3120
    """
    if pd.isna(x):
        return np.nan
    s = str(x).strip()
    # remove R$, espaços e quaisquer símbolos não numéricos (mantém '.' e ',')
    s = re.sub(r'[^\d,.\-]', '', s)
    # se possui milhares com ponto e decimais com vírgula → troca padrão BR pra padrão US
    if s.count(',') == 1 and s.rfind(',') > s.rfind('.'):
        s = s.replace('.', '').replace(',', '.')
    else:
        s = s.replace(',', '.')
    try:
        return float(s)
    except:
        return np.nan

def _normalize_manual_table(df_manual: pd.DataFrame) -> pd.DataFrame | None:
    """
    Normaliza a tabela manual:
      - Colunas para UPPER
      - DATA_ROTA -> DATA (date)
      - Converte CUSTO_FROTA_AGREGADA para float
      - Mantém apenas colunas necessárias
      - Remove duplicados por (DATA, IDVEICULO), mantendo a última ocorrência
    """
    if df_manual is None:
        return None

    # padroniza nomes
    dfm = df_manual.copy()
    dfm.columns = [c.strip().upper() for c in dfm.columns]

    # tenta mapear a coluna de data
    if "DATA" in dfm.columns:
        pass
    elif "DATA_ROTA" in dfm.columns:
        dfm.rename(columns={"DATA_ROTA": "DATA"}, inplace=True)
    else:
        st.warning("A Tabela Manual não possui coluna 'DATA' ou 'DATA_ROTA'. Ignorando substituição do custo agregado.")
        return None

    required = {"DATA", "IDVEICULO", "CUSTO_FROTA_AGREGADA"}
    if not required.issubset(dfm.columns):
        st.warning(f"Tabela Manual não contém as colunas esperadas: {sorted(list(required))}. Ignorando substituição do custo agregado.")
        return None


    # normaliza DATA para date
    dfm["DATA"] = pd.to_datetime(dfm["DATA"], errors="coerce").dt.date

    # normaliza custo para float
    dfm["CUSTO_FROTA_AGREGADA"] = dfm["CUSTO_FROTA_AGREGADA"].apply(_money_to_float)

    # mantém apenas colunas necessárias e remove linhas com DATA/IDVEICULO inválidos ou custo NaN
    dfm = dfm[["DATA", "IDVEICULO", "CUSTO_FROTA_AGREGADA"]].dropna(subset=["DATA", "IDVEICULO", "CUSTO_FROTA_AGREGADA"])

    # remove duplicidades por DATA+IDVEICULO (mantém a última)
    dfm = dfm.sort_values(["DATA", "IDVEICULO"]).drop_duplicates(subset=["DATA", "IDVEICULO"], keep="last")

    return dfm


# Modified OSRM function to return polyline
def osrm_distance_duration_polyline(coords):
    if len(coords)<2: return 0,0,None
    base = "https://router.project-osrm.org/route/v1/driving/"
    path = ";".join([f"{c[0]:.6f},{c[1]:.6f}" for c in coords])
    url = f"{base}{path}?overview=full&geometries=polyline6"
    try:
        r = requests.get(url, timeout=20)
        if r.status_code == 200:
            data = r.json()
            if data.get("routes"):
                route = data["routes"][0]
                dist_km = route["distance"] / 1000.0
                dur_h = route["duration"] / 3600.0
                poly6 = route.get("geometry")  # polyline for the toll API
                return dist_km, dur_h, poly6
    except Exception:
        pass
    return 0.0, 0.0, None


def vehicle_class(p):
    if p<=600: return "UTILITARIO_600"
    elif p<=1600: return "VAN_1600"
    elif p<=3000: return "TRES_QUARTOS_3000"
    elif p<=5000: return "SEMI_TOCO_5000"
    elif p<=14000: return "TRUCK"
    else: return "CARRETA"

def map_col(v):
    return {
        "UTILITARIO_600":"FRETE UTILITÁRIO 600Kg",
        "VAN_1600":"FRETE VAN 1600Kg",
        "TRES_QUARTOS_3000":"FRETE 3/4 3000Kg",
        "SEMI_TOCO_5000":"FRETE SEMI - TOCO 5000Kg",
        "TRUCK":"FRETE TRUCK",
        "CARRETA":"FRETE CARRETA"
    }.get(v)

def interpolate(df, km_col, val_col, km):
    if val_col not in df.columns: return np.nan
    df=df.sort_values(km_col)
    x=df[km_col].astype(float).values; y=df[val_col].astype(float).values
    if km<=x.min(): return y[0]
    if km>=x.max(): return y[-1]
    i=np.searchsorted(x,km)
    x0,x1=x[i-1],x[i]; y0,y1=y[i-1],y[i]
    return y0+(y1-y0)*(km-x0)/(x1-x0)

# New function to map vehicle class to axles
def eixo_por_classe(vclass):
    # adjust according to your real fleet
    return {
        "UTILITARIO_600": 2,
        "VAN_1600": 2,
        "TRES_QUARTOS_3000": 2,
        "SEMI_TOCO_5000": 3,
        "TRUCK": 3,
        "CARRETA": 5,   # can be 5–9; adjust according to trailer
    }.get(vclass, 2)

# New function to call TollGuru API (provided by user)
def toll_cost_for_route_tollguru(polyline6, vehicle_axles, api_key=None):
    """
    Calcula pedágio via TollGuru a partir de uma polilinha (polyline6) e eixos do veículo.
    Retorna custo total em BRL ou 0.0 se falhar.
    """
    if not api_key or not polyline6:
        return 0.0
    url = "https://dev.tollguru.com/v1/calc/route"
    headers = {"x-api-key": api_key, "Content-Type": "application/json"}
    payload = {
        "source": "OSRM",
        "polyline": polyline6,
        "vehicleType": "truck",        # ou "car" se aplicável
        "vehicle": {
            "axles": vehicle_axles
        },
        "country": "BR", # Changed to fixed "BR"
        "currency": "BRL"
    }
    try:
        resp = requests.post(url, headers=headers, json=payload, timeout=30)
        if resp.status_code == 200:
            data = resp.json()
            # verifique o caminho do total (varia por versão); exemplo comum:
            total = (
                data.get("route", {})
                    .get("costs", {})
                    .get("tag", {})
                    .get("currency", "BRL")
            )
            # fallback direto:
            total_val = data.get("route", {}).get("costs", {}).get("tag", {}).get("amount")
            if total_val is None:
                # outro formato possível
                total_val = data.get("route", {}).get("costs", {}).get("cash", {}).get("amount", 0.0)
            return float(total_val or 0.0)
        else:
             st.warning(f"TollGuru API returned status code: {resp.status_code} - {resp.text}")
             return 0.0
    except Exception as e:
        st.warning(f"Error calling TollGuru API: {e}")
        pass
    return 0.0


st.set_page_config(page_title="Comparador de Custos", layout="wide")
# Add the image logo
logo_path = "/content/logo-branca-qfpsii0s3y2kjypwqu57rvsp16k4hj6dc0rz3dj1ia.png"

if os.path.exists(logo_path):
    st.image(logo_path, use_container_width=False)
else:
    st.warning("⚠️ Logo image not found. Faça upload do arquivo 'logo-branca-qfpsii0s3y2kjypwqu57rvsp16k4hj6dc0rz3dj1ia.png' no diretório /content/.")

st.title("🚛 Análise Comparativa das Modalidades Operacionais de Transporte: Frota Própria, Agregado e Cross-Docking")

with st.sidebar:
    st.header("⚙️ Parâmetros")
    total_fixo_per_day = 147.36 # Modified variable name with new value
    st.write("**Custo fixo total/dia:**",round(total_fixo_per_day,2)) # Used new variable name
    km_l=st.number_input("Consumo (km/L)",5.0)
    preco_comb=st.number_input("Preço Combustível",5.99)

    # Added selectbox for toll method
    pedagio_provider = st.selectbox("Pedágio - Método", ["manual (taxa Rá/km)", "TollGuru API"])
    per_km_toll = st.number_input("Taxa Pedágio (R$/km)", min_value=0.0, value=0.45, step=0.05, format="%.2f")
    tollguru_key = st.text_input("TollGuru API Key", type="password")


    j_ini,j_fim=6,16
    # Removed custo_h_extra input as it will be calculated
    base_lat=st.number_input("Base Lat",-23.603)
    base_lon=st.number_input("Base Lon",-46.919)

    st.subheader("Uploads")
    cli=st.file_uploader("RelatorioCadastroCliente")
    rota=st.file_uploader("rota")
    tab=st.file_uploader("TabelaFreteCustosFrotaAgregada")
    cross=st.file_uploader("CustosCrossDocking")
    # Added new file uploader for manual table
    tabela_manual = st.file_uploader("Tabela Manual (Substituir Custo Agregado)")
    run=st.button("🚀 Executar")

def read_df(f):
    if f is None: return None
    n=f.name.lower()
    if n.endswith(".csv"): return pd.read_csv(f)
    return pd.read_excel(f)

if run:
    df_cli,df_rota,df_tab,df_cross,df_tabela_manual=[read_df(x) for x in [cli,rota,tab,cross,tabela_manual]] # Added df_tabela_manual
    if any(df is None for df in [df_cli,df_rota,df_tab,df_cross]):
        st.error("Faltam planilhas obrigatórias!"); st.stop() # Modified error message
    if "TollGuru" in pedagio_provider and not tollguru_key: # Check if API key is provided when TollGuru is selected
        st.error("TollGuru API Key é necessária para calcular pedágios."); st.stop()

    df_cli.rename(columns=lambda x:x.strip(),inplace=True)
    df_rota.rename(columns=lambda x:x.strip().upper(),inplace=True)
    df_tab.rename(columns=lambda x:x.strip(),inplace=True)
    df_cross.rename(columns=lambda x:x.strip().upper(),inplace=True)
    df_cli_m=df_cli[["CódigoCliente","Latitude","Longitude","TempoMédioEntrega", "Segmento"]] # Included Segmento column
    df_cli_m.columns=["IDCLI","LAT","LON","TEMPO", "SEGMENTO"] # Renamed Segmento column
    df=df_rota.merge(df_cli_m,on="IDCLI",how="left")

    resultados=[]
    detalhe_frota_propria = [] # New list for detailed own fleet cost
    detalhe_tempo_jornada = [] # New list for detailed journey time
    base=(base_lon,base_lat)
    custo_motorista_fixo = 164.25 # Fixed driver cost
    custo_indireto_fixo = 772.10 # Fixed indirect cost
    custo_ajudante_fixo = 127.53 # Fixed helper cost

    # Calculate number of unique vehicles before the loop
    num_unique_vehicles = df_rota['IDVEICULO'].nunique()
    # Calculate the divided indirect cost
    custo_indireto_dividido = custo_indireto_fixo / num_unique_vehicles if num_unique_vehicles > 0 else 0

    # --- localizar coluna de data na planilha rota ---
    def _detect_date_column(df_rota: pd.DataFrame):
        candidates = ["DATA", "DATA_ROTA", "DATA_ENTREGA", "DT_CARGA", "DATA_SAIDA"]
        for c in candidates:
            if c in df_rota.columns:
                return c
        # fallback: tenta achar por dtype datetime
        for c in df_rota.columns:
            if np.issubdtype(df_rota[c].dtype, np.datetime64):
                return c
        return None

    date_col = _detect_date_column(df_rota)
    if date_col is None:
        st.warning("⚠️ Não encontrei coluna de data em 'rota'. Adicionando data padrão (hoje) para auditoria.")
        df_rota["_DATA_AUD"] = pd.Timestamp("today").normalize()
        date_col = "_DATA_AUD"
    else:
        # normaliza para date (sem hora)
        df_rota[date_col] = pd.to_datetime(df_rota[date_col], errors="coerce").dt.date


    # Change grouping to process by both date and vehicle ID
    for (current_date, vid), g in df.groupby([date_col, "IDVEICULO"]):
        coords=[base]+[(float(x.LON),float(x.LAT)) for _,x in g.iterrows()]+[base]
        km,hr, polyline = osrm_distance_duration_polyline(coords) # Updated to receive polyline
        tempo=g["TEMPO"].apply(to_hours).sum()
        total_h=hr+tempo
        extra=max(0,total_h-(j_fim-j_ini))
        # Calculate Hora trabalhada and Custo Hora Extra based on MDO
        custo_motorista = custo_motorista_fixo
        # Use the divided indirect cost
        custo_indireto = custo_indireto_dividido
        custo_ajudante = custo_ajudante_fixo # Added helper cost

        # Check if any client in the route has "Segmento" as "Atacado"
        if "Atacado" in g["SEGMENTO"].values:
            custo_ajudante = 0 # Set custo_ajudante to 0 if any client is Atacado


        # Check if number of unique clients is greater than 1 and double helper cost (only if not Atacado)
        num_unique_clients = g["IDCLI"].nunique()
        if num_unique_clients > 1 and "Atacado" not in g["SEGMENTO"].values:
            custo_ajudante = custo_ajudante * 2


        total_custo_mdo = custo_motorista + custo_ajudante # Calculate total MDO cost
        hora_trabalhada_rate = total_custo_mdo / 9 if total_custo_mdo > 0 else 0 # Calculate Hora trabalhada rate, avoid division by zero
        custo_extra = extra * hora_trabalhada_rate * 1.40 # Calculate Custo Hora Extra with 40% premium

        comb=(km/km_l)*preco_comb
        # Calculate toll cost based on selected method
        peso=g["PESO"].sum()
        vcl=vehicle_class(peso)
        axles = eixo_por_classe(vcl) # Get axles based on vehicle class

        if "TollGuru" in pedagio_provider:
            ped = toll_cost_for_route_tollguru(polyline, axles, tollguru_key) # Calculate toll cost using API
        else:
            ped = km * per_km_toll # Calculate toll cost using manual rate

        # aqui é o calculo de pedágio
        ped = ped * 2 # Multiply toll cost by 2 as requested
        ped = ped + (axles * 1.0) # Add 1 real per axle

        frota_propria=total_fixo_per_day+comb+ped+custo_extra+custo_motorista+custo_indireto+custo_ajudante # Included all fixed costs
        col=map_col(vcl)
        km_ida=km/2
        frete=interpolate(df_tab,"KM IDA",col,km_ida) if col else np.nan

        cross_v=peso * 0.38
        # Append results using the date and vid from the group key
        resultados.append([current_date, vid, round(km,1),peso,frota_propria,frete,cross_v,vcl, num_unique_clients])

        # Append detailed own fleet cost for this vehicle, including all fixed costs and total MDO
        detalhe_frota_propria.append([current_date, vid, total_fixo_per_day, comb, ped, custo_extra, custo_motorista, custo_indireto, custo_ajudante, total_custo_mdo, frota_propria])

        # Append detailed journey time for this vehicle
        detalhe_tempo_jornada.append([current_date, vid, tempo, hr, total_h, extra])


    out=pd.DataFrame(resultados,columns=["DATA", "IDVEICULO","KM_TOTAL","PESO","CUSTO_FROTA_PRORIA","CUSTO_FROTA_AGREGADA","CUSTO_CROSS","VEICULO", "NUM_UNIQUE_CLIENTS"]) # Added DATA


    # =========================================================
    # SUBSTITUIR CUSTO_FROTA_AGREGADA USANDO "Tabela Manual (Substituir Custo Agregada)"
    # Requer colunas: DATA (ou DATA_ROTA), IDVEICULO, CUSTO_FROTA_AGREGADA
    # =========================================================

    if tabela_manual is not None:
        try:
            # Use read_df to handle both excel and csv
            df_manual_raw = read_df(tabela_manual)
        except Exception as e:
            st.warning(f"Erro ao ler 'Tabela Manual (Substituir Custo Agregada)': {e}")
            df_manual_raw = None

        df_manual_ok = _normalize_manual_table(df_manual_raw)

        if df_manual_ok is not None and not df_manual_ok.empty:
            st.info(f"Aplicando {len(df_manual_ok)} substituições de CUSTO_FROTA_AGREGADA da Tabela Manual...")
            # garante que 'out' tem a coluna DATA como date e IDVEICULO como string para merge
            out["DATA"] = pd.to_datetime(out["DATA"], errors="coerce").dt.date
            out["IDVEICULO"] = out["IDVEICULO"].astype(str)

            # merge para trazer custo manual como coluna auxiliar
            # df_manual_ok is indexed by (DATA, IDVEICULO)
            # Reset index of df_manual_ok so DATA and IDVEICULO become columns for merging
            df_manual_ok_reset = df_manual_ok.reset_index()

            # Ensure 'DATA' and 'IDVEICULO' in the reset manual table are ready for merging
            df_manual_ok_reset["DATA"] = pd.to_datetime(df_manual_ok_reset["DATA"], errors="coerce").dt.date
            df_manual_ok_reset['IDVEICULO'] = df_manual_ok_reset['IDVEICULO'].astype(str) # Ensure IDVEICULO is string


            # merge out with the processed manual table on 'DATA' and 'IDVEICULO' columns
            out = out.merge(
                df_manual_ok_reset[['DATA', 'IDVEICULO', 'CUSTO_FROTA_AGREGADA']].rename(columns={'CUSTO_FROTA_AGREGADA': 'CUSTO_AGREGADA_MANUAL'}),
                on=["DATA", "IDVEICULO"], # Merge on columns now
                how="left"
            )

            # substitui CUSTO_FROTA_AGREGADA pelo manual quando disponível
            out["CUSTO_FROTA_AGREGADA"] = np.where(
                out["CUSTO_AGREGADA_MANUAL"].notna(),
                out["CUSTO_AGREGADA_MANUAL"],
                out["CUSTO_FROTA_AGREGADA"]
            )

            # remove a auxiliar
            out.drop(columns=["CUSTO_AGREGADA_MANUAL"], inplace=True)

            st.success("CUSTO_FROTA_AGREGADA substituído com base na Tabela Manual quando aplicável.")
        elif df_manual_ok is not None and df_manual_ok.empty:
             st.info("Tabela Manual carregada, mas vazia ou sem dados válidos para substituição.")
        else:
             st.warning("Tabela Manual não processada corretamente. Substituição não aplicada.")
    else:
        st.info("Nenhuma Tabela Manual (Substituir Custo Agregado) carregada. Usando custos calculados.")

    # Recalculate MODAL_MAIS_BARATO after applying the manual override
    out["MODAL_MAIS_BARATO"]=out[["CUSTO_FROTA_PRORIA","CUSTO_FROTA_AGREGADA","CUSTO_CROSS"]].idxmin(axis=1)

    st.dataframe(out)


    # Display the second table for detailed own fleet cost
    st.subheader("Detalhe do Custo da Frota Propria") # Subheader for the new table
    df_detalhe = pd.DataFrame(detalhe_frota_propria, columns=["DATA", "IDVEICULO", "Custo Fixo Diário", "Custo Combustível", "Custo Pedágio", "Custo Hora Extra", "Custo Motorista", "Custo ADM", "Custo Ajudante", "Total Custo MDO", "Total Frota Propria"]) # New DataFrame, added DATA
    st.dataframe(df_detalhe) # Display the new DataFrame

    # Display the third table for detailed journey time
    st.subheader("Detalhe do Tempo de Jornada") # Subheader for the third table
    df_tempo = pd.DataFrame(detalhe_tempo_jornada, columns=["DATA", "IDVEICULO", "Tempo Médio Entrega (horas)", "Tempo de Trajeto Estimado (horas)", "Total Horas Jornada", "Total de Hora Extra"]) # New DataFrame, added DATA
    st.dataframe(df_tempo) # Display the new DataFrame

    # Create and display the new table for Operational Economy Analysis
    st.subheader("Análise de Economia Operacional")
    # Ensure 'DATA' column in out is date objects for consistent merging
    out['DATA'] = pd.to_datetime(out['DATA'], errors='coerce').dt.date
    # Merge out with df_rota to get NOMETRANSPORTADORACRIACAO
    # Use only necessary columns from df_rota for merging to avoid conflicts
    # Use 'out' which now contains the potentially overridden CUSTO_FROTA_AGREGADA
    df_economia = out.merge(df_rota[["IDVEICULO", date_col, "NOMETRANSPORTADORACRIACAO"]].drop_duplicates(subset=["IDVEICULO", date_col]), left_on=["IDVEICULO", "DATA"], right_on=["IDVEICULO", date_col], how="left") # Merge on DATA and IDVEICULO

    df_economia["MENOR_MODAL"] = df_economia[["CUSTO_FROTA_PRORIA", "CUSTO_FROTA_AGREGADA"]].idxmin(axis=1)
    df_economia["MENOR_CUSTO"] = df_economia[["CUSTO_FROTA_PRORIA", "CUSTO_FROTA_AGREGADA"]].min(axis=1)

    # Determine CUSTO_ESCOLHIDO based on NOMETRANSPORTADORACRIACAO
    df_economia["CUSTO_ESCOLHIDO"] = df_economia.apply(
        lambda row: row["NOMETRANSPORTADORACRIACAO"] if pd.isna(row["NOMETRANSPORTADORACRIACAO"]) else (row["CUSTO_FROTA_PRORIA"] if row["NOMETRANSPORTADORACRIACAO"] == "PROPRIO" else row["CUSTO_FROTA_AGREGADA"]),
        axis=1
    )
    # convert CUSTO_ESCOLHIDO to numeric, coercing errors to NaN
    df_economia["CUSTO_ESCOLHIDO"] = pd.to_numeric(df_economia["CUSTO_ESCOLHIDO"], errors='coerce')


    df_economia["DIFERENCA_R$"] = df_economia["CUSTO_ESCOLHIDO"] - df_economia["MENOR_CUSTO"]
    df_economia["DIFERENCA_%"] = np.where(df_economia["MENOR_CUSTO"] > 0, df_economia["DIFERENCA_R$"] / df_economia["MENOR_CUSTO"] * 100.0, np.nan)


    # Drop the NOMETRANSPORTADORACRIACAO and original date_col columns from the merged result if they exist and are not the final 'DATA' column
    cols_to_drop = ["NOMETRANSPORTADORACRIACAO"]
    # Check if date_col exists in df_economia before dropping
    if date_col in df_economia.columns and date_col != "DATA":
         cols_to_drop.append(date_col)
    df_economia = df_economia.drop(columns=cols_to_drop, errors='ignore')


    # Reorder columns to have DATA and IDVEICULO at the beginning
    economia_cols = ["DATA", "IDVEICULO"] + [col for col in df_economia.columns if col not in ["DATA", "IDVEICULO"]]
    # Ensure all columns in economia_cols are actually in df_economia before reordering
    economia_cols = [col for col in economia_cols if col in df_economia.columns]
    df_economia = df_economia[economia_cols]


    # Format currency and percentage columns
    # Ensure columns exist and are numeric before formatting
    for col in ["CUSTO_FROTA_PRORIA", "CUSTO_FROTA_AGREGADA", "MENOR_CUSTO", "CUSTO_ESCOLHIDO", "DIFERENCA_R$", "DIFERENCA_%"]:
        if col in df_economia.columns:
             # Convert to numeric first, coercing errors
            df_economia[col] = pd.to_numeric(df_economia[col], errors='coerce')
            if col == "DIFERENCA_%":
                 df_economia[col] = df_economia[col].map('{:.2f}%'.format, na_action='ignore')
            else:
                 df_economia[col] = df_economia[col].map('{:.2f}'.format, na_action='ignore')



    st.dataframe(df_economia)


    # ============================
    # AUDITORIA – 5 DIAS (BLOCO)
    # ============================

    # --- garantir scipy/statsmodels para p-valores e ICs (instala se faltar) ---
    def _ensure_stats_libs():
        try:
            import scipy, statsmodels  # noqa
            return True
        except Exception:
            try:
                subprocess.check_call([sys.executable, "-m", "pip", "install", "-U", "scipy", "statsmodels"])
                return True
            except Exception as e:
                st.warning(f"Não foi possível instalar scipy/statsmodels automaticamente: {e}")
                return False

    _stats_ok = _ensure_stats_libs()
    if _stats_ok:
        import scipy.stats as stats
    else:
        stats = None

    # --- construir base por IDVEICULO (1 linha por carga) com DATA e modal escolhido ---
    # pega a 1ª data por veículo, e o modal escolhido informado
    meta_rota = (
        df_rota[["IDVEICULO", date_col, "NOMETRANSPORTADORACRIACAO"]]
        .dropna(subset=["IDVEICULO"])
        .drop_duplicates(subset=["IDVEICULO", date_col]) # Include date_col in drop_duplicates
        .rename(columns={date_col: "DATA"})
    )

    # junta com a tabela 'out' (que já tem os custos por IDVEICULO)
    # Ensure 'DATA' column in meta_rota is date objects for consistent merging
    meta_rota['DATA'] = pd.to_datetime(meta_rota['DATA'], errors='coerce').dt.date
    audit = out.merge(meta_rota, on=["IDVEICULO", "DATA"], how="left") # Merge on DATA and IDVEICULO

    # --- mapeia modal escolhido para custo escolhido (C_ESC) ---
    def _map_modal_to_cost(row):
        modal_raw = (row.get("NOMETRANSPORTADORACRIACAO") or "").strip().upper()
        if modal_raw in ["PROPRIO", "PRÓPRIO", "FROTA PROPRIA", "FROTA PRÓPRIA", "PROPRIA", "PRIVADO"]:
            return row["CUSTO_FROTA_PRORIA"]
        if modal_raw in ["AGREGADO", "TERCEIRO", "FROTA AGREGADA"]:
            return row["CUSTO_FROTA_AGREGADA"]
        if modal_raw in ["CROSS", "CROSSDOCKING", "CD", "CROSS-DOCKING"]:
            return row["CUSTO_CROSS"]
        # fallback: se não souber, assume custo agregado (para não deixar NaN)
        return row["CUSTO_FROTA_AGREGADA"]

    audit["C_ESC"] = audit.apply(_map_modal_to_cost, axis=1)

    # --- menor custo técnico e modal ótimo ---
    audit["C_MIN"] = audit[["CUSTO_FROTA_PRORIA", "CUSTO_FROTA_AGREGADA", "CUSTO_CROSS"]].min(axis=1)
    audit["MODAL_MIN"] = audit[["CUSTO_FROTA_PRORIA", "CUSTO_FROTA_AGREGADA", "CUSTO_CROSS"]].idxmin(axis=1)

    # --- sucesso (acerto) e perdas ---
    audit["SUCCESS"] = (audit["C_ESC"] == audit["C_MIN"]).astype(int)
    audit["DELTA_R$"] = audit["C_ESC"] - audit["C_MIN"]
    audit["DELTA_%"] = np.where(audit["C_MIN"] > 0, audit["DELTA_R$"] / audit["C_MIN"] * 100.0, np.nan)

    # --- limitar para os 5 dias mais recentes com base em 'DATA' ---
    if "DATA" in audit.columns and audit["DATA"].notna().any():
        # ordena datas e pega top 5 distintas
        last5 = (
            pd.Series(sorted(audit["DATA"].dropna().unique()))
            .sort_values()
            .tail(5)
            .tolist()
        )
        audit_5d = audit[audit["DATA"].isin(last5)].copy()
    else:
        # se não houver datas, usa tudo como uma janela fictícia
        audit_5d = audit.copy()
        audit_5d["DATA"] = pd.Timestamp("today").date()

    # ==============================
    # TABELAS DA AUDITORIA – 5 DIAS
    # ==============================

    st.subheader("🔎 Auditoria – 5 dias (acertos, perdas e testes)")

    # 1) Tabela por carga (janela 5d)
    cols_order = [
        "DATA","IDVEICULO",
        "CUSTO_FROTA_PRORIA","CUSTO_FROTA_AGREGADA","CUSTO_CROSS",
        "C_ESC","C_MIN",
        "NOMETRANSPORTADORACRIACAO","MODAL_MIN","SUCCESS","DELTA_R$","DELTA_%"
    ]
    cols_present = [c for c in cols_order if c in audit_5d.columns]
    st.markdown("**Tabela por carga (janela 5 dias)**")
    st.dataframe(audit_5d[cols_present].sort_values(["DATA","IDVEICULO"]))

    # 2) Agregados diários
    def _agg_day(g: pd.DataFrame):
        n = len(g)
        acc = g["SUCCESS"].mean() * 100.0 if n else np.nan
        perda_media = g["DELTA_R$"].mean() if n else np.nan
        perda_total = g["DELTA_R$"].sum() if n else np.nan
        mediana = g["DELTA_R$"].median() if n else np.nan
        q25 = g["DELTA_R$"].quantile(0.25) if n else np.nan
        q75 = g["DELTA_R$"].quantile(0.75) if n else np.nan
        return pd.Series({
            "n_cargas": n,
            "Acc_dia_%": acc,
            "Perda_Média_Carga_R$": perda_media,
            "Perda_Total_R$": perda_total,
            "Mediana_Delta_R$": mediana,
            "p25_Delta_R$": q25,
            "p75_Delta_R$": q75
        })

    by_day = audit_5d.groupby("DATA", dropna=False).apply(_agg_day).reset_index()
    st.markdown("**Agregados diários (5 dias)**")
    st.dataframe(by_day.sort_values("DATA"))

    # 3) Sumário 5 dias
    total_cargas = len(audit_5d)
    acc_micro = audit_5d["SUCCESS"].mean() * 100.0 if total_cargas else np.nan
    acc_macro = by_day["Acc_dia_%"].mean() if len(by_day) else np.nan
    acc_macro_sd = by_day["Acc_dia_%"].std(ddof=1) if len(by_day) > 1 else 0.0
    perda_total_5d = audit_5d["DELTA_R$"].sum()
    perda_media_carga = audit_5d["DELTA_R$"].mean() if total_cargas else np.nan
    perda_media_pct = audit_5d["DELTA_%"].mean() if total_cargas else np.nan

    kpi = pd.DataFrame({
        "Acc_micro_%":[acc_micro],
        "Acc_macro_%":[acc_macro],
        "DesvPad_Acc_dia":[acc_macro_sd],
        "Perda_total_5d_R$":[perda_total_5d],
        "Perda_média_por_carga_R$":[perda_media_carga],
        "Perda_média_%":[perda_media_pct]
    })
    st.markdown("**KPIs – janela 5 dias**")
    st.dataframe(kpi)

    # ==================
    # TESTES ESTATÍSTICOS
    # ==================
    st.markdown("### 🧪 Testes estatísticos")

    # -- t pareado (DELTA_R$ vs 0) --
    delta_vals = audit_5d["DELTA_R$"].dropna().values
    t_res, p_res, ci_low, ci_high, cohend = None, None, None, None, None
    if len(delta_vals) >= 2 and stats is not None:
        m = float(np.mean(delta_vals))
        s = float(np.std(delta_vals, ddof=1))
        n = len(delta_vals)
        t_stat = m / (s / sqrt(n)) if s > 0 else np.inf
        df = n - 1
        # unicaudal (H1: média > 0)
        p_val = 1 - stats.t.cdf(t_stat, df)
        # IC 95% bilateral
        t_crit = stats.t.ppf(0.975, df)
        ci_low = m - t_crit * s / sqrt(n)
        ci_high = m + t_crit * s / sqrt(n)
        cohend = m / s if s > 0 else np.inf
        t_res, p_res = t_stat, p_val

        st.write(f"**t pareado (ΔR$ > 0):** t = {t_stat:.3f}, df = {df}, p = {p_val:.4f}")
        st.write(f"**IC 95% para média(ΔR$):** [{ci_low:.2f}, {ci_high:.2f}] | **Cohen's d** = {cohend:.3f}")
    elif len(delta_vals) < 2:
        st.info("Amostra insuficiente para t-teste (precisa de pelo menos 2 cargas com ΔR$).")
    else:
        st.info("Biblioteca estatística ausente; tente novamente (o app tentou instalar scipy).")

    # -- teste binomial de acerto (micro): H0: p = p0 (ex.: 0.5) --
    p0 = 0.5
    k_success = int(audit_5d["SUCCESS"].sum())
    n_tot = int(total_cargas)
    if n_tot >= 1 and stats is not None:
        # aproximação normal para binomial (quando n grande) ou usar statsmodels/scipy se desejado
        try:
            from statsmodels.stats.proportion import proportion_confint
            ci_lo, ci_hi = proportion_confint(k_success, n_tot, alpha=0.05, method="beta")  # Clopper–Pearson
        except Exception:
            # Wilson manual (fallback)
            z = 1.959963984540054
            phat = k_success / n_tot if n_tot else 0.0
            denom = 1 + z**2 / n_tot
            center = phat + z*z/(2*n_tot)
            half = z*sqrt((phat*(1-phat)+z*z/(4*n_tot))/n_tot)
            ci_lo = (center - half)/denom
            ci_hi = (center + half)/denom

    # ==================
    # GRÁFICOS (Altair)
    # ==================
    try:
        # 1) Linha: acurácia por dia
        chart_acc = alt.Chart(by_day.assign(DATA=by_day["DATA"].astype(str))).mark_line(point=True).encode(
            x=alt.X('DATA:N', title='Data'),
            y=alt.Y('Acc_dia_%:Q', title='Acurácia do dia (%)')
        ).properties(title='Acurácia diária (5 dias)')
        st.altair_chart(chart_acc, use_container_width=True)

        # 2) Barras: perda total por dia
        chart_perda = alt.Chart(by_day.assign(DATA=by_day["DATA"].astype(str))).mark_bar().encode(
            x=alt.X('DATA:N', title='Data'),
            y=alt.Y('Perda_Total_R$:Q', title='Perda total (R$)')
        ).properties(title='Perda total por dia (R$)')
        st.altair_chart(chart_perda, use_container_width=True)

        # 3) Boxplot: ΔR$ por dia
        chart_box = alt.Chart(audit_5d.assign(DATA=audit_5d["DATA"].astype(str))).mark_boxplot().encode(
            x=alt.X('DATA:N', title='Data'),
            y=alt.Y('DELTA_R$:Q', title='ΔC = C_ESC - C_MIN (R$)')
        ).properties(title='Distribuição de ΔR$ por dia')
        st.altair_chart(chart_box, use_container_width=True)
    except Exception as e:
        st.warning(f"Não foi possível renderizar os gráficos Altair: {e}")


    # Export all dataframes to different sheets in one Excel file
    output = io.BytesIO()
    with pd.ExcelWriter(output, engine='xlsxwriter') as writer:
        out.to_excel(writer, sheet_name='Comparativo Custos', index=False)
        df_detalhe.to_excel(writer, sheet_name='Detalhe Frota Propria', index=False)
        df_tempo.to_excel(writer, sheet_name='Detalhe Tempo Jornada', index=False)
        df_economia.to_excel(writer, sheet_name='Analise Economia Operacional', index=False)

        # >>> NOVAS ABAS – AUDITORIA 5 DIAS <<<
        try:
            audit_5d[cols_present].to_excel(writer, sheet_name='Auditoria_5d_Cargas', index=False)
        except Exception:
            audit_5d.to_excel(writer, sheet_name='Auditoria_5d_Cargas', index=False)
        by_day.to_excel(writer, sheet_name='Auditoria_5d_Diario', index=False)
        kpi.to_excel(writer, sheet_name='KPIs_5d', index=False)


    output.seek(0)

    st.download_button(
        label="Baixar Excel Completo",
        data=output,
        file_name="Comparativo_Custos_Transporte_Completo.xlsx",
        mime="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet"
    )

else:
    st.info("Carregue planilhas e clique em Executar")
'''

launcher_code = r'''
import os, subprocess, time, sys
from pyngrok import ngrok
from PIL import Image # Import Pillow for image handling

def ensure_installed():
    import pkgutil
    pkgs=["streamlit","pyngrok","pandas","numpy","requests","xlsxwriter","Pillow"] # Added Pillow
    need=[p for p in pkgs if pkgutil.find_loader(p) is None]
    if need: subprocess.check_call([sys.executable,"-m","pip","install",*need])

def run():
    port=8501
    token=os.environ.get("NGROK_AUTH_TOKEN","")
    if token: ngrok.set_auth_token(token)

    # Disconnect any existing ngrok tunnels
    try:
        tunnels = ngrok.get_tunnels()
        if tunnels:
            print("Disconnecting existing tunnels...")
            for tunnel in tunnels:
                print(f"  Disconnecting {tunnel.public_url}")
                ngrok.disconnect(tunnel.public_url)
    except Exception as e:
        print(f"Error disconnecting tunnels: {e}")

    url=ngrok.connect(port).public_url
    print("🌐 Public URL:",url)
    p=subprocess.Popen(["streamlit","run","app.py"])
    try:
        while True: time.sleep(5)
    except KeyboardInterrupt: p.terminate()

if __name__=="__main__":
    ensure_installed(); run()
'''

with open("app.py","w",encoding="utf-8") as f: f.write(app_code)
with open("run_in_colab.py","w",encoding="utf-8") as f: f.write(launcher_code)

print("✅ Arquivos criados: app.py e run_in_colab.py")

# === 2️⃣ Instala dependências e executa Streamlit com ngrok ===
!pip install streamlit pyngrok pandas numpy requests xlsxwriter Pillow -q # Added Pillow

import os
os.environ["NGROK_AUTH_TOKEN"] = "33vthvkGJREkZID7YROACMe1BMT_KWnW3rpre6rUhGJQV282"  # <- Substitua aqui pelo seu token ngrok

!python run_in_colab.py

✅ Arquivos criados: app.py e run_in_colab.py
/content/run_in_colab.py:9: DeprecationWarning: 'pkgutil.find_loader' is deprecated and slated for removal in Python 3.14; use importlib.util.find_spec() instead
  need=[p for p in pkgs if pkgutil.find_loader(p) is None]
🌐 Public URL: https://sammy-cybernetic-denice.ngrok-free.dev



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://136.114.250.233:8501

2025-12-31 18:27:43.001 Please replace `use_container_width` with `width`.

`use_container_width` will be removed after 2025-12-31.

For `use_container_width=True`, use `width='stretch'`. For `use_container_width=False`, use `width='content'`.
2025-12-31 18:28:57.835 Please replace `use_container_width` with `width`.

`use_container_width` will be removed after 2025-12-31.

For `use_container_width=True`, use `width='stretch'`. For `use_container_width=False`, use `width='content'`.
2025-12